# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [7]:
import numpy as np
import math
import pandas as pd
import yfinance as yf
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [8]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,A,Agilent Technologies,Health Care,Life Sciences Tools & Services,"Santa Clara, California",05/06/00,1090872,1999
1,AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals","Cupertino, California",30/11/82,320193,1977
2,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",31/12/12,1551152,2013 (1888)
3,ABNB,Airbnb,Consumer Discretionary,"Hotels, Resorts & Cruise Lines","San Francisco, California",18/09/23,1559720,2008
4,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",04/03/57,1800,1888
...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",01/11/11,1524472,2011
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",06/10/97,1041061,1997
500,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",07/08/01,1136869,1927
501,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",23/12/19,877212,1969


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [14]:
symbol = 'AAPL'

data = yf.Ticker(symbol).history(period='1y')

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [15]:
start_price = data['Close'].iloc[0]
close_price = data['Close'].iloc[-1]

year_return = (close_price / start_price) - 1
print(year_return)

0.21931005699464245


## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [35]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Symbol'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(' '.join(symbol_groups[i]))
    

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [39]:
final_df = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call = yf.download(symbol_string, period="1y")['Close']

    year_returns = ((batch_api_call.iloc[-1] - batch_api_call.iloc[0]) / batch_api_call.iloc[0])

    for symbol in symbol_string.split(' '):
        price = batch_api_call[symbol].iloc[-1]
        new_row = pd.DataFrame([[symbol, price, year_returns[symbol], 'N/A']], columns=my_columns)

        final_df = pd.concat([final_df, new_row], ignore_index=True)
final_df    

[*********************100%***********************]  100 of 100 completed

1 Failed download:
['AMTM']: YFInvalidPeriodError("%ticker%: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']")
/var/folders/hd/h4ty4f0j2tgfss4vb7rrr93w0000gn/T/ipykernel_81848/943318742.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat([final_df, new_row], ignore_index=True)
[*********************100%***********************]  100 of 100 completed
[*********************100%***********************]  100 of 100 completed
[*********************100%***********************]  100 of 100 completed
[*********************100%***********************]  100 of 100 completed

1 Failed download:
['SW']: YFInvalidPeriodError("%t

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,133.669998,0.243905,N/A
1,AAPL,224.229996,0.213366,N/A
2,ABBV,171.089996,0.234059,N/A
3,ABNB,133.199997,0.117919,N/A
4,ABT,116.440002,0.215576,N/A
...,...,...,...,...
498,XYL,124.790001,0.266775,N/A
499,YUM,134.610001,0.074216,N/A
500,ZBH,109.940002,0.038934,N/A
501,ZBRA,402.690002,0.987513,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [44]:
final_df.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_df = final_df[:50]
final_df.reset_index(inplace=True)
final_df

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,476,VST,142.160004,3.132558,N/A
1,346,NVDA,148.289993,2.049979,N/A
2,376,PLTR,59.849998,2.036530,N/A
3,47,AXON,595.179993,1.721943,N/A
4,393,RCL,237.419998,1.427359,N/A
5,269,KKR,153.160004,1.394247,N/A
6,458,UAL,89.129997,1.338756,N/A
7,188,FICO,2348.100098,1.319638,N/A
8,446,TRGP,194.309998,1.288423,N/A
9,236,HWM,113.580002,1.256257,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [50]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio: ')

    try:
        float(portfolio_size)
    except ValueError:
        print('Please enter an numerical value')
        portfolio_size = input('Enter the size of your portfolio: ')

portfolio_input()

In [55]:
position_size = float(portfolio_size)/len(final_df.index)
for i in range(0, len(final_df)):
    final_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_df.loc[i, 'Price'])
final_df

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,476,VST,142.160004,3.132558,1406
1,346,NVDA,148.289993,2.049979,1348
2,376,PLTR,59.849998,2.036530,3341
3,47,AXON,595.179993,1.721943,336
4,393,RCL,237.419998,1.427359,842
5,269,KKR,153.160004,1.394247,1305
6,458,UAL,89.129997,1.338756,2243
7,188,FICO,2348.100098,1.319638,85
8,446,TRGP,194.309998,1.288423,1029
9,236,HWM,113.580002,1.256257,1760


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [117]:
hqm_cols = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]
hqm_df = pd.DataFrame(columns=hqm_cols)

for symbol_string in symbol_strings:
    batch_api_call = yf.download(symbol_string, period="1y")['Close']

    one_year_change = ((batch_api_call.iloc[-1] - batch_api_call.iloc[0]) / batch_api_call.iloc[0])
    six_month_change = ((batch_api_call.iloc[-1] - batch_api_call.iloc[-(252//2)]) / batch_api_call.iloc[-(252//2)])
    three_month_change = ((batch_api_call.iloc[-1] - batch_api_call.iloc[-(252//4)]) / batch_api_call.iloc[-(252//4)])
    one_month_change = ((batch_api_call.iloc[-1] - batch_api_call.iloc[-(252//12)]) / batch_api_call.iloc[-(252//12)])

    for symbol in symbol_string.split(' '):
        price = batch_api_call[symbol].iloc[-1]
        new_row = pd.DataFrame([[symbol, price, 'N/A', one_year_change[symbol], 'N/A', six_month_change[symbol], 'N/A', three_month_change[symbol], 'N/A', one_month_change[symbol], 'N/A', 'N/A']], columns=hqm_cols)

        hqm_df = pd.concat([hqm_df, new_row], ignore_index=True)
hqm_df

[*********************100%***********************]  100 of 100 completed

1 Failed download:
['AMTM']: YFInvalidPeriodError("%ticker%: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']")
/var/folders/hd/h4ty4f0j2tgfss4vb7rrr93w0000gn/T/ipykernel_81848/1589053098.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  hqm_df = pd.concat([hqm_df, new_row], ignore_index=True)
[*********************100%***********************]  100 of 100 completed
[*********************100%***********************]  100 of 100 completed
[*********************100%***********************]  100 of 100 completed
[*********************100%***********************]  100 of 100 completed

1 Failed download:
['SW']: YFInvalidPeriodError("%tick

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,126.919998,N/A,0.113529,N/A,-0.174074,N/A,-0.093364,N/A,-0.071951,N/A,N/A
1,AAPL,228.020004,N/A,0.191016,N/A,0.185443,N/A,0.007156,N/A,-0.035775,N/A,N/A
2,ABBV,166.279999,N/A,0.202227,N/A,0.020561,N/A,-0.153921,N/A,-0.108609,N/A,N/A
3,ABNB,132.190002,N/A,0.019198,N/A,-0.082651,N/A,0.123300,N/A,-0.036446,N/A,N/A
4,ABT,117.360001,N/A,0.159684,N/A,0.139860,N/A,0.053595,N/A,0.003163,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
498,XYL,122.459999,N/A,0.222034,N/A,-0.160946,N/A,-0.098233,N/A,-0.075913,N/A,N/A
499,YUM,134.009995,N/A,0.045973,N/A,-0.037976,N/A,-0.027151,N/A,0.000075,N/A,N/A
500,ZBH,113.019997,N/A,0.004622,N/A,-0.050252,N/A,0.007937,N/A,0.074335,N/A,N/A
501,ZBRA,384.279999,N/A,0.742450,N/A,0.194232,N/A,0.113855,N/A,0.031375,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [118]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_df.index:
    for tp in time_periods:
        change_col = f'{tp} Price Return'
        percentile_col = f'{tp} Return Percentile'
        hqm_df[change_col] = hqm_df[change_col].fillna(0)
        hqm_df.loc[row, percentile_col] = stats.percentileofscore(hqm_df[change_col], hqm_df.loc[row, change_col]) / 100

hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,126.919998,N/A,0.113529,0.337972,-0.174074,0.093439,-0.093364,0.157058,-0.071951,0.206759,N/A
1,AAPL,228.020004,N/A,0.191016,0.455268,0.185443,0.707753,0.007156,0.38171,-0.035775,0.326044,N/A
2,ABBV,166.279999,N/A,0.202227,0.483101,0.020561,0.341948,-0.153921,0.079523,-0.108609,0.109344,N/A
3,ABNB,132.190002,N/A,0.019198,0.214712,-0.082651,0.188867,0.123300,0.713718,-0.036446,0.32008,N/A
4,ABT,117.360001,N/A,0.159684,0.409543,0.139860,0.620278,0.053595,0.512922,0.003163,0.489066,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
498,XYL,122.459999,N/A,0.222034,0.508946,-0.160946,0.101392,-0.098233,0.149105,-0.075913,0.194831,N/A
499,YUM,134.009995,N/A,0.045973,0.244533,-0.037976,0.246521,-0.027151,0.284294,0.000075,0.471173,N/A
500,ZBH,113.019997,N/A,0.004622,0.206759,-0.050252,0.230616,0.007937,0.387674,0.074335,0.775348,N/A
501,ZBRA,384.279999,N/A,0.742450,0.942346,0.194232,0.723658,0.113855,0.67992,0.031375,0.602386,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [119]:
from statistics import mean

for row in hqm_df.index:
    momentum_percentiles = []
    for tp in time_periods:
        momentum_percentiles.append(hqm_df.loc[row, f'{tp} Return Percentile'])
    hqm_df.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,126.919998,N/A,0.113529,0.337972,-0.174074,0.093439,-0.093364,0.157058,-0.071951,0.206759,0.198807
1,AAPL,228.020004,N/A,0.191016,0.455268,0.185443,0.707753,0.007156,0.38171,-0.035775,0.326044,0.467694
2,ABBV,166.279999,N/A,0.202227,0.483101,0.020561,0.341948,-0.153921,0.079523,-0.108609,0.109344,0.253479
3,ABNB,132.190002,N/A,0.019198,0.214712,-0.082651,0.188867,0.123300,0.713718,-0.036446,0.32008,0.359344
4,ABT,117.360001,N/A,0.159684,0.409543,0.139860,0.620278,0.053595,0.512922,0.003163,0.489066,0.507952
...,...,...,...,...,...,...,...,...,...,...,...,...
498,XYL,122.459999,N/A,0.222034,0.508946,-0.160946,0.101392,-0.098233,0.149105,-0.075913,0.194831,0.238569
499,YUM,134.009995,N/A,0.045973,0.244533,-0.037976,0.246521,-0.027151,0.284294,0.000075,0.471173,0.31163
500,ZBH,113.019997,N/A,0.004622,0.206759,-0.050252,0.230616,0.007937,0.387674,0.074335,0.775348,0.400099
501,ZBRA,384.279999,N/A,0.742450,0.942346,0.194232,0.723658,0.113855,0.67992,0.031375,0.602386,0.737078


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [120]:
hqm_df.sort_values('HQM Score', ascending=False, inplace=True)
hqm_df = hqm_df[:50]
hqm_df.reset_index(inplace=True, drop=True)
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,PLTR,61.259998,N/A,1.870665,0.998012,1.884181,1.0,0.882606,0.998012,0.434660,0.998012,0.998509
1,AXON,600.000000,N/A,1.652285,0.994036,1.083695,0.998012,0.599573,0.992048,0.351412,0.996024,0.99503
2,UAL,90.379997,N/A,1.258371,0.99006,0.704318,0.992048,1.152417,1.0,0.221351,0.982107,0.991054
3,TRGP,199.000000,N/A,1.290516,0.992048,0.683587,0.99006,0.398356,0.976143,0.230598,0.986083,0.986083
4,RCL,231.759995,N/A,1.190341,0.984095,0.537380,0.980119,0.465722,0.982107,0.159322,0.946322,0.973161
5,VST,146.630005,N/A,3.226867,1.0,0.555921,0.984095,0.782086,0.996024,0.124636,0.900596,0.970179
6,TPR,56.549999,N/A,0.838426,0.964215,0.335301,0.926441,0.374241,0.972167,0.270501,0.992048,0.963718
7,DFS,173.220001,N/A,0.992179,0.976143,0.383767,0.944334,0.298890,0.940358,0.182631,0.972167,0.95825
8,SYF,64.050003,N/A,1.147887,0.980119,0.429688,0.962227,0.355843,0.964215,0.147644,0.924453,0.957753
9,FICO,2272.310059,N/A,1.161121,0.982107,0.652829,0.988072,0.297277,0.93837,0.139317,0.918489,0.956759


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [121]:
portfolio_input()

In [122]:
position_size = float(portfolio_size) / len(hqm_df.index)
for i in hqm_df.index:
    hqm_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_df.loc[i, 'Price'])
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,PLTR,61.259998,326,1.870665,0.998012,1.884181,1.0,0.882606,0.998012,0.434660,0.998012,0.998509
1,AXON,600.000000,33,1.652285,0.994036,1.083695,0.998012,0.599573,0.992048,0.351412,0.996024,0.99503
2,UAL,90.379997,221,1.258371,0.99006,0.704318,0.992048,1.152417,1.0,0.221351,0.982107,0.991054
3,TRGP,199.000000,100,1.290516,0.992048,0.683587,0.99006,0.398356,0.976143,0.230598,0.986083,0.986083
4,RCL,231.759995,86,1.190341,0.984095,0.537380,0.980119,0.465722,0.982107,0.159322,0.946322,0.973161
5,VST,146.630005,136,3.226867,1.0,0.555921,0.984095,0.782086,0.996024,0.124636,0.900596,0.970179
6,TPR,56.549999,353,0.838426,0.964215,0.335301,0.926441,0.374241,0.972167,0.270501,0.992048,0.963718
7,DFS,173.220001,115,0.992179,0.976143,0.383767,0.944334,0.298890,0.940358,0.182631,0.972167,0.95825
8,SYF,64.050003,312,1.147887,0.980119,0.429688,0.962227,0.355843,0.964215,0.147644,0.924453,0.957753
9,FICO,2272.310059,8,1.161121,0.982107,0.652829,0.988072,0.297277,0.93837,0.139317,0.918489,0.956759


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [128]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_df.to_excel(writer, sheet_name="Momentum Strategy", index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [129]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [130]:
col_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template]
}

for col in col_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{col}:{col}', 22, col_formats[col][1])
    writer.sheets['Momentum Strategy'].write(f'{col}1', col_formats[col][0], col_formats[col][1])

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [131]:
writer.close()